# Marine EOV Broker



In [1]:
from lib.MarineRiBroker import MarineBroker
import logging

logger = logging.getLogger()
logger.setLevel(logging.INFO)

print(MarineBroker.ERDDAP_OUTPUT_FORMATS)
print(MarineBroker.EOV_LIST)

['csv', 'geoJson', 'json', 'nc', 'ncCF', 'odvTxt']
['EV_OXY', 'EV_SEATEMP', 'EV_SALIN', 'EV_CURR', 'EV_CHLA', 'EV_CO2', 'EV_NUTS']


## Start the broker

It will take some time (though it still needs improvements on performances). This is because the broker will :
* load vocabularies upon startup
* load erddap datasets metadata from all erddap servers


**Question :**
Do we want to work with all datasets on Erddap servers ? Or do we want to build a fixed list for them ?

In [2]:
%%time
broker = MarineBroker()

INFO:root:Querying vocabulary server for EOV : EV_OXY
INFO:root:Querying vocabulary server for EOV : EV_SEATEMP
INFO:root:Querying vocabulary server for EOV : EV_SALIN
INFO:root:Querying vocabulary server for EOV : EV_CURR
INFO:root:Querying vocabulary server for EOV : EV_CHLA
INFO:root:Querying vocabulary server for EOV : EV_CO2
INFO:root:Querying vocabulary server for EOV : EV_NUTS


CPU times: user 3.31 s, sys: 352 ms, total: 3.66 s
Wall time: 17.3 s


## Create a request to the broker :
The user must provide the EOVs, min/max date/lat/lon, output format desired.

When creating a query, the broker :
* first looks at every dataset to see if they match any eov requested by the user
* then checks if the datasets match the time/bbox requested by the user

In [3]:
%%time
# logger.setLevel(logging.DEBUG)
queries = broker.submit_request(["EV_SALIN", "EV_OXY", "EV_SEATEMP", "EV_CO2", "EV_CHLA"], "2020-01-01", "2020-01-31", -10, 30, 30, 60, "csv")

CPU times: user 303 ms, sys: 73.5 ms, total: 376 ms
Wall time: 6.29 s


## Results

The interesting part !
The broker provides the result as a list of python objects defined in the broker API.

In [4]:
queries

In [5]:
[q.query_url for q in queries]

['https://www.ifremer.fr/erddap/tabledap/ArgoFloats.csv?time%2Clatitude%2Clongitude%2Cpsal%2Cdoxy%2Ctemp&time%3E=2020-01-01&time%3C=2020-01-31&latitude%3E=30&latitude%3C=60&longitude%3E=-10&longitude%3C=30',
 'https://www.ifremer.fr/erddap/tabledap/ArgoFloats-synthetic-BGC.csv?time%2Clatitude%2Clongitude%2Cpsal%2Cbisulfide%2Ccdom%2Cdoxy%2Cmlpl_doxy%2Cmolar_nitrate%2Ctemp&time%3E=2020-01-01&time%3C=2020-01-31&latitude%3E=30&latitude%3C=60&longitude%3E=-10&longitude%3C=30']

### Looking inside an ErddapRequest object :


In [6]:
sample_query = queries[0]
print(sample_query.query_url)
print(sample_query.dataset_name)
print(sample_query.query_variables)


https://www.ifremer.fr/erddap/tabledap/ArgoFloats.csv?time%2Clatitude%2Clongitude%2Cpsal%2Cdoxy%2Ctemp&time%3E=2020-01-01&time%3C=2020-01-31&latitude%3E=30&latitude%3C=60&longitude%3E=-10&longitude%3C=30
ArgoFloats
['psal', 'doxy', 'temp']


### Showing all urls

In [7]:
for q in queries:
    print(q.query_url)

https://www.ifremer.fr/erddap/tabledap/ArgoFloats.csv?time%2Clatitude%2Clongitude%2Cpsal%2Cdoxy%2Ctemp&time%3E=2020-01-01&time%3C=2020-01-31&latitude%3E=30&latitude%3C=60&longitude%3E=-10&longitude%3C=30
https://www.ifremer.fr/erddap/tabledap/ArgoFloats-synthetic-BGC.csv?time%2Clatitude%2Clongitude%2Cpsal%2Cbisulfide%2Ccdom%2Cdoxy%2Cmlpl_doxy%2Cmolar_nitrate%2Ctemp&time%3E=2020-01-01&time%3C=2020-01-31&latitude%3E=30&latitude%3C=60&longitude%3E=-10&longitude%3C=30


### Convenience method to retrieve data in xarray

We can see that erddap keeps the variables attributes in the output data.

Global attributes could be adapted in the to_xarray method in order to provide the proper min/max values.

In [8]:
ds = sample_query.to_xarray()
ds

<xarray.Dataset>
Dimensions:          (trajectory: 80, profile: 563, obs: 241069)
Coordinates:
    time             (profile) datetime64[ns] ...
    latitude         (profile) float64 ...
    longitude        (profile) float64 ...
    pres             (obs) float32 ...
Dimensions without coordinates: trajectory, profile, obs
Data variables:
    platform_number  (trajectory) object ...
    cycle_number     (profile) float64 ...
    trajectoryIndex  (profile) int32 ...
    rowSize          (profile) int32 ...
    psal             (obs) float32 ...
    doxy             (obs) float32 ...
    temp             (obs) float32 ...
Attributes: (12/36)
    cdm_altitude_proxy:        pres
    cdm_data_type:             TrajectoryProfile
    cdm_profile_variables:     cycle_number, data_type, format_version, handb...
    cdm_trajectory_variables:  platform_number, project_name, pi_name, platfo...
    Conventions:               Argo-3.1, CF-1.6, COARDS, ACDD-1.3
    creator_email:             support@argo.net
    ...                        ...
    summary:                   Argo float vertical profiles from Coriolis Glo...
    time_coverage_end:         2020-01-30T22:52:30Z
    time_coverage_start:       2020-01-01T05:02:30Z
    title:                     Argo Float Measurements
    user_manual_version:       3.1
    Westernmost_Easting:       -9.754393333333333

### Same but to get a pandas dataframe

In [9]:
df = sample_query.to_pandas_dataframe()
df

/tmp/ipykernel_24718/211131177.py:1: DtypeWarning: Columns (1,2,3,4,5) have mixed types.Specify dtype option on import or set low_memory=False.
  df = sample_query.to_pandas_dataframe()


,time,latitude,longitude,psal,doxy,temp
0,UTC,degrees_north,degrees_east,PSU,micromole/kg,degree_Celsius
1,2020-01-05T05:32:30Z,44.40783166666667,-2.898385,35.075,NaN,13.525
2,2020-01-05T05:32:30Z,44.40783166666667,-2.898385,35.073,NaN,13.525
3,2020-01-05T05:32:30Z,44.40783166666667,-2.898385,35.074,NaN,13.526
4,2020-01-05T05:32:30Z,44.40783166666667,-2.898385,35.073,NaN,13.528
...,...,...,...,...,...,...
241065,2020-01-26T05:22:00Z,41.886285,28.993265,22.291,NaN,8.971
241066,2020-01-26T05:22:00Z,41.886285,28.993265,22.294,NaN,8.976
241067,2020-01-26T05:22:00Z,41.886285,28.993265,22.297,NaN,8.98
241068,2020-01-26T05:22:00Z,41.886285,28.993265,22.301,NaN,8.985


## To be done :
* optimizations

## Other ideas :

* download the data on disk ?